In [ ]:
#| hide
from metricwalk.core import *

## Installation
Currently, MetricWalk can only be installed from source.

### Prerequisites
You will need

- Python3
- Networkx
- Numpy
- Gensim
- Joblib

I highly recommend installing an [Anaconda](https://www.anaconda.com/distribution/#download-section) environment. Future versions of MetricWalk will be available on PyPI and conda.

## Background
- [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/abs/1301.3781)
- [DeepWalk: Online Learning of Social Representations](https://arxiv.org/abs/1403.6652)
- [node2vec: Scalable Feature Learning for Networks](https://arxiv.org/abs/1607.00653)
- [On the Surprising Behaviour of node2vec](https://arxiv.org/abs/2206.08252)
- Forthcoming paper by [Tesfa Asmara](http://tesfaasmara.com)

## Notation
Let us establish some notation used in sections below:

- Vertex set is $V = \lbrace 1,2,...,x,y,...,N \rbrace$ where $x$ and $y$ are some generic vertices of interest
- Edge set is $E = \lbrace (x,y):x,y \in V \rbrace$
- Graph $G$ is defined by a pair $(V,E)$
- An adjacency matrix of $G$ is a matrix $A=\lbrack a_{xy} \rbrack_{N \times N}$ such that $a_{xy}=1$ if $(x,y) \in E$ otherwise $a_{xy}=0$.
- A set of neighbors of vertex $x$ in graph $G$ is $N(x)$, namely $N(x)=\lbrace y:(x,y) \in E \rbrace$. Let $\lvert N(x)\rvert$ denote the number of elements in the set of neighbors $N(x)$.

## Random Walks

Formally, we define a spatial graph embedding method that simulates a random walk of fixed length $l$ starting at a given source node $u$. Let $c_{i}$ denote the $i^{th}$ node in the walk, where $c_{0} = u$. The nodes $c_{i}$ are generated by the following distribution:

$$P(c_i =x|c_{i-1} =y)= \begin{cases}
			\frac{\pi_{xy}}{Z}, & \text{if $(x,y) \in E$}\\
            0, & \text{otherwise}
		 \end{cases}
$$

where $\pi_xy$ is the unnormalized transition probability between nodes $x$ and $y$, and $Z$ is the normalizing constant.


## Search Bias $\pi$

When biasing random walks using different indexes, the choice of index can influence the search procedure to explore different types of network neighborhoods. Choice indexes include the Common Neighbors Index, Salton Index, Jaccard Index, Sorenson Index, Hub Promoted Index, Hub Depressed Index, Leicht-Holme-Newman Index, Preferential Attachment Index, Adamic-Adar Index, and Resource Allocation Index. These indexes each measure the similarity or strength of the connection between vertices $x$ and $y$, and assign weights accordingly to guide the random walk towards the next node. The choice of index should be based on the specific task and the nature of the network, as each index emphasizes different aspects of the network structure. 

1. Common Neighbors 
$$\pi_{xy} = \lvert N(x) \cap N(y) \rvert$$

This index measures the number of common neighbors between vertices $x$ and $y$. The higher the number of common neighbors, the more likely the random walker is to move from vertex $x$ to vertex $y$, as they share a strong connection through their common neighbors.

2. Salton Index
$$\pi_{xy} = \frac{\lvert N(x) \cap N(y) \rvert}{\sqrt{\lvert N(x) \rvert \times \lvert N(y) \rvert}}$$

This index measures the cosine similarity between the sets of neighbors of vertices $x$ and $y$. The higher the cosine similarity, the more likely the random walker is to move from vertex $x$ to vertex $y$, as they share a similar set of neighbors.

3. Jaccard Index
$$\pi_{xy} = \frac{\lvert N(x) \cap N(y) \rvert}{\lvert N(x) \cup N(y) \rvert}$$

This index measures the proportion of common neighbors between vertices $x$ and $y$ relative to the total number of neighbors they have. The higher the proportion of common neighbors, the more likely the random walker is to move from vertex $x$ to vertex $y$, as they share a strong connection through their common neighbors relative to their total neighbors.

4. Sorenson Index
$$\pi_{xy} = \frac{2 \lvert N(x) \cap N(y) \rvert}{\lvert N(x) \rvert + \lvert N(y) \rvert}$$

This index is similar to the Jaccard index, but it takes into account the size of the individual sets of neighbors. The higher the intersection between the sets of neighbors of vertices $x$ and $y$ relative to their total number of neighbors, the more likely the random walker is to move from vertex $x$ to vertex $y$.

5. Hub Promoted Index
$$\pi_{xy} = \frac{\lvert N(x) \cap N(y) \rvert}{\min \lbrace \lvert N(x) \rvert, \lvert N(y) \rvert \rbrace}$$

This index measures the strength of the connection between vertices $x$ and $y$ relative to the size of their respective sets of neighbors. The higher the intersection between the sets of neighbors of vertices $x$ and $y$ relative to the size of the smaller set, the more likely the random walker is to move from vertex $x$ to vertex $y$, as they have a strong connection relative to the size of their sets of neighbors.

6. Hub Depressed Index
$$\pi_{xy} = \frac{\lvert N(x) \cap N(y) \rvert}{\max \lbrace \lvert N(x) \rvert, \lvert N(y) \rvert \rbrace}$$

This index is the opposite of the Hub Promoted Index. The higher the intersection between the sets of neighbors of vertices $x$ and $y$ relative to the size of the larger set, the less likely the random walker is to move from vertex $x$ to vertex $y$, as they have a weak connection relative to the size of their sets of neighbors.

7. Leicht-Holme-Newman Index
$$\pi_{xy} = \frac{\lvert N(x) \cap N(y) \rvert}{\lvert N(x) \rvert \times \lvert N(y) \rvert}$$

This index measures the probability of finding a common neighbor between vertices $x$ and $y$ relative to the product of their degrees. The higher the probability, the more likely the random walker is to move from vertex $x$ to vertex $y$, as they have a strong connection relative to their degrees.

8. Preferential Attachment
$$\pi_{xy} = \lvert N(x) \rvert \times \lvert N(y) \rvert$$ 

This index measures the product of the degrees of vertices $x$ and $y$. The higher the product, the more likely the random walker is to move from vertex $x$ to vertex $y$, as they have a high degree.

9. Adamic–Adar Index
$$\pi_{xy} = \sum_{z \in N(x) \cap N(y)} \frac{1}{\log \lvert N(z) \rvert}$$

This index assigns higher weights to common neighbors with lower degrees, as they are considered more important in measuring the similarity between vertices $x$ and $y$. The higher the score, the more likely the random walker is to move from vertex $x$ to vertex $y$, as they have a strong connection through their common neighbors with lower degrees.

10. Resource Allocation Index
$$\pi_{xy} = \sum_{z \in N(x) \cap N(y)} \frac{1}{\lvert N(z) \rvert}$$

This index measures the amount of resources that would be transmitted between vertices $x$ and $y$ through their common neighbors. The higher the score, the more likely the random walker is to move from vertex $x$ to vertex $y$, as they have a strong connection through their common neighbors. 

11. Maximum Flow 
$$\pi_{xy} = \texttt{maxFlow}(x,y)$$

The maximum flow between two vertices $x$ and $y$ is the maximum amount of flow that can be sent from $x$ to $y$ through the edges of the graph, subject to certain capacity constraints on the edges. The maximum flow is a measure of the maximum amount of information or resources that can be transferred from $x$ to $y$. The higher the maximum flow the more likely the random walker is to move towards edges that can transmit more information or resources between the two vertices.

12. Minimum-Cost Maximum-Flow
$$\pi_{xy} = \texttt{minCostMaxFlow}(x,y)$$

This measure considers the cost of sending flow through the edges, in addition to the capacity constraints. The minimum-cost maximum flow between two vertices $x$ and $y$ is the maximum flow that can be sent from $x$ to $y$ subject to capacity constraints, where the cost of sending flow through the edges is minimized. The cost of sending flow through an edge can be thought of as a measure of the “distance” or “effort” required to transfer information or resources through that edge.

13. Levenshtein distance
$$\pi_{xy} = 1 - \texttt{lev}(N(x),N(y))$$

14. Tversky index
$$\pi_{xy} = \frac{\lvert N(x) \cap N(y) \rvert}{\lvert N(x) \cup N(y) \rvert + \alpha \lvert N(x) \setminus N(y)\rvert + \beta \lvert N(y) \setminus N(x)\rvert}$$


Given a `MetricWalk` object, these functions can be passed into `MetricWalk.fit()` as `MetricWalk.<func>` where `<func>` is one of `jaccard_coefficient`, `adamic_adar`, `resource_allocation`, `common_neighbors`, `sorenson_index`, `salton_index`, `hub_depressed`, `hub_promoted`, `preferential_attachment`, `lhn_index`, `max_flow`, and `max_flow_min_cost`.

## Caveats
- Node names in the input graph must be all strings, or all ints
- Parallel execution not working on Windows (joblib known issue). To run non-parallel on Windows pass `workers=1` in the MetricWalk’s constructor.
- Note that `max_flow` and `max_flow_min_cost` have a runtime of $O(N^4 \sqrt{E})$; parallelization has been incorporated where applicable throughout to alleviate the expense of this methodology.
- Moreover, in order to use the `max_flow` and `max_flow_min_cost` transformations, you must specify a `capacity` attribute on the edges of your Networkx graph. You can do this by setting the capacity of the edges equal to the weights of the edges or some other value like so:

In [ ]:
import networkx as nx

# Create a graph
graph = nx.fast_gnp_random_graph(n=100, p=0.5)

# Set the capacity of the edges equal to 1
for u, v in graph.edges:
    # or consider setting this value equal to graph[u][v]['weight'], if it exists for your graph.
    graph[u][v]['capacity'] = 1

## Parameters
- `MetricWalk` constructor:
    * `graph`: The first positional argument has to be a networkx graph. Node names must be all integers or all strings. On the output model they will always be strings.
    * `dimensions`: Embedding dimensions (default: 128)
    * `walk_length`: Number of nodes in each walk (default: 80)
    * `num_walks`: Number of walks per node (default: 10)
    * `workers`: Number of workers for parallel execution (default: 1)
- `MetricWalk.fit` method: Returns a trained `gensim.Word2Vec` model

## How to use

In [ ]:
import networkx as nx
from metricwalk.core import MetricWalk

# Create a graph
graph = nx.fast_gnp_random_graph(n=100, p=0.5)

# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
metricWalk = MetricWalk(graph, dimensions = 128, window = 10, walk_length = 80, num_walks = 10, workers = 1)

# Embed nodes using preferential attachment
model = metricWalk.fit(metricWalk.preferential_attachment)

# Look for most similar nodes
model.wv.most_similar(2) # Output node names are always strings

# Save embeddings for later use
# model.wv.save_word2vec_format(EMBEDDING_FILENAME)

# Save model for later use
# model.save(EMBEDDING_MODEL_FILENAME)